In [1]:
import pandas as pd
import geopandas
import datetime
from geopandas import GeoDataFrame
from shapely.geometry import Point

In [3]:
#import fire calls into pandas
fireCalls = pd.read_csv("Seattle_Real_Time_Fire_911_Calls.csv")

#remove unnecessary initial line
fireCalls = fireCalls.drop([0, 0])

#add columns to break down data further
#https://erikrood.com/Python_References/extract_month_year_pandas_final.html
fireCalls['Month'] = pd.DatetimeIndex(fireCalls['Datetime']).month
fireCalls['Year'] = pd.DatetimeIndex(fireCalls['Datetime']).year
fireCalls['Neighborhood'] = ''

In [5]:
#import seattle neighborhood shapefile
seaNH = geopandas.read_file("City_Clerk_Neighborhoods\City_Clerk_Neighborhoods.shp")

#need to remove excess rows at beginning for extra islands
seaNH = seaNH.drop(seaNH.index[:26])

In [6]:
#get seaNH projection data to use for fire calls geodata frame 
seaNH_crs = seaNH.crs

#convert fireCalls to geodataframe
#https://gis.stackexchange.com/questions/174159/convert-a-pandas-dataframe-to-a-geodataframe
geometry = [Point(xy) for xy in zip(fireCalls.Longitude, fireCalls.Latitude)]
crs = seaNH_crs
fc_geo = GeoDataFrame(fireCalls, crs = crs, geometry = geometry)

In [12]:
#spatial join fc_geo to neighborhoods so each fire call has a neighborhood attribute
seaFC = geopandas.sjoin(fc_geo, seaNH, how = 'inner', op = 'within')

In [11]:
seaFC.to_file('seattlefc')